In [1]:
# TODO: some of these are not needed, remove unused imports
import numpy as np
from scipy import sparse
import os
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from joblib import dump, load

In [2]:
def load_validation_set(dir):
    replays = []

    for path in os.listdir(dir):
        full_path = os.path.join(dir, path)
        print(full_path)
        if os.path.isfile(full_path):
            replays.append(np.asarray(sparse.load_npz(full_path).todense()))

    Y = []
    X = []
    for replay in replays:
        Y.append(replay[0, 0])
        X.append(replay[:, 1:])

    Y = np.array(Y)
    X = np.array(X)

    return X, Y

In [3]:
x_val, y_val = load_validation_set(
    "/Users/mathiaskirkbonde/Documents/parsed_replays/GlobalFeatureVector/Terran_vs_Terran/Terran/validation/"
)

5d4a3.SC2Replay.npz
/Users/mathiaskirkbonde/Documents/parsed_replays/GlobalFeatureVector/Terran_vs_Terran/Terran/validation/2@d27204ce4007d53c07533ff0f366f25905913f7ba1e64683b895b34d0f779d23.SC2Replay.npz
/Users/mathiaskirkbonde/Documents/parsed_replays/GlobalFeatureVector/Terran_vs_Terran/Terran/validation/1@f02217c8ff0de290acdf9e99c619ec6aaa44c1b644810d6db6d9781fe314721b.SC2Replay.npz
/Users/mathiaskirkbonde/Documents/parsed_replays/GlobalFeatureVector/Terran_vs_Terran/Terran/validation/2@ef8dcafdfcbf46f9fe0c89e9b240900797a19668d42f0aaeb46e9f3e38a1250c.SC2Replay.npz
/Users/mathiaskirkbonde/Documents/parsed_replays/GlobalFeatureVector/Terran_vs_Terran/Terran/validation/1@6da92631cfad5b4f2bc009efbf9ed794c470f43d7bfc8695751391f7ec1de587.SC2Replay.npz
/Users/mathiaskirkbonde/Documents/parsed_replays/GlobalFeatureVector/Terran_vs_Terran/Terran/validation/2@b69def6a4874d7528563dc19a31a23d91d878e2d65e9f082204041255094c0d3.SC2Replay.npz
/Users/mathiaskirkbonde/Documents/parsed_replays/Global

In [4]:
random_forest = load("./models/random_forest.joblib")
decision_tree = load("./models/decision_tree.joblib")

In [5]:
val_acc = {
    "random_forest": [],
    "decision_tree": [],
    }

Benchmarking classical models from the last timestep to 100 timesteps back

In [6]:
for timestep in range(1, 100):
    X = []
    for replay in x_val:
        X.append(replay[-timestep, :])

    X = np.array(X)
    score = (random_forest.score(X, y_val),
            decision_tree.score(X, y_val))
    val_acc["random_forest"].append(score[0])
    val_acc["decision_tree"].append(score[1])

Benchmarking the LSTM model

In [7]:
import json

with open("./models/classical_benchmarks.json", "w") as f:
    f.write(json.dumps(val_acc))

In [8]:
from tensorflow import keras

def load_keras_models(dir):
    models = []
    val_acc = {}
    for model in os.listdir(dir):
        models.append((keras.models.load_model(dir + model), model))
        val_acc[model] = []
    return models, val_acc

lstm_models, lstm_val_acc = load_keras_models('./models/lstm_models/')
#lstm_model_dropout = keras.models.load_model('./models/model.savedmodel/')


Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x15a4f48e0> and <tensorflow.python.keras.layers.core.Dense object at 0x15a553820>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x15b8bd460> and <tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x15b8c0430>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x15b8c0430> and <tensorflow.python.keras.layers.core.Dense object at 0x15b8c55b0>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x15dd1da90> and <tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x15dd27a60>).

Two checkpoint references resolved to different objects (<tensorflow.python.keras.layers.recurrent_v2.LSTM object at 0x15dd27a60> and <tensorflow.python.keras.layers.recurre

Benchmarking LSTM's on final step, best performing models are benchmarked on full timeseries.
The reasoning for this is that my way of benchmarking is super ineffecient. A better LSTM architecture that returns sequences would benchmark at 100x the speed.

In [9]:
X = []
for replay in x_val:
    X.append(replay[-102:-1, :])

X = np.array(X)
for model in lstm_models:
    print(model)
    lstm_val_acc[model[1]].append(1 - model[0].evaluate(X, y_val, batch_size=1))
#print(no_dropout_model.evaluate(X, y_val, batch_size=1))

(<tensorflow.python.keras.engine.sequential.Sequential object at 0x15a4af910>, '32_x1_lstm.savedmodel')
978/978 [==============================] - 4s 4ms/step - loss: 0.2209
(<tensorflow.python.keras.engine.sequential.Sequential object at 0x15b89c910>, '64x2_lstm.savedmodel')
978/978 [==============================] - 9s 9ms/step - loss: 0.4551
(<tensorflow.python.keras.engine.sequential.Sequential object at 0x15dd16a60>, '32x4_lstm.savedmodel')
978/978 [==============================] - 22s 22ms/step - loss: 0.1625
(<tensorflow.python.keras.engine.sequential.Sequential object at 0x15ea042b0>, '32_x1_0.2dropout_lstm.savedmodel')
978/978 [==============================] - 8s 8ms/step - loss: 0.2494
(<tensorflow.python.keras.engine.sequential.Sequential object at 0x160b67d60>, '32x4_0.2dropoutx3lstm.savedmodel')
978/978 [==============================] - 30s 31ms/step - loss: 0.4171
(<tensorflow.python.keras.engine.sequential.Sequential object at 0x1625e0580>, '32x3_lstm.savedmodel')
978

In [10]:
best_models = []
cut_off = 0.80

for model in lstm_val_acc:
    if cut_off < lstm_val_acc[model][0]:
        print(f"{model}: ", lstm_val_acc[model][0])
        best_models.append(model)

32x4_lstm.savedmodel:  0.8374648094177246
32_x2_lstm.savedmodel:  0.8897008895874023
32_x2_0.2dropout_lstm.savedmodel:  0.8369124382734299


In [11]:
# padding replays
x_val = keras.preprocessing.sequence.pad_sequences(x_val)

Benchmarking models that made the cutoff of 80% accuracy at last step.

In [12]:
for timestep in range(2, 300):
    X = []
    for replay in x_val:
        X.append(replay[-timestep-101:-timestep, :])

    X = np.array(X)
    
    print(timestep)
    for model in lstm_models:
        if model[1] in best_models:
            print(model[1])
            lstm_val_acc[model[1]].append(1 - model[0].evaluate(X, y_val, batch_size=1))
    if timestep % 2 == 0:
        with open("./models/lstm_benchmarks.json", "w") as f:
            f.write(json.dumps(lstm_val_acc))

- loss: 0.4234
32_x2_lstm.savedmodel
978/978 [==============================] - 8s 8ms/step - loss: 0.4023
32_x2_0.2dropout_lstm.savedmodel
978/978 [==============================] - 8s 8ms/step - loss: 0.4280
233
32x4_lstm.savedmodel
978/978 [==============================] - 15s 15ms/step - loss: 0.4247
32_x2_lstm.savedmodel
978/978 [==============================] - 8s 8ms/step - loss: 0.4069
32_x2_0.2dropout_lstm.savedmodel
978/978 [==============================] - 8s 8ms/step - loss: 0.4278
234
32x4_lstm.savedmodel
978/978 [==============================] - 15s 16ms/step - loss: 0.4207
32_x2_lstm.savedmodel
978/978 [==============================] - 8s 8ms/step - loss: 0.4095
32_x2_0.2dropout_lstm.savedmodel
978/978 [==============================] - 8s 9ms/step - loss: 0.4276
235
32x4_lstm.savedmodel
978/978 [==============================] - 16s 17ms/step - loss: 0.4259
32_x2_lstm.savedmodel
978/978 [==============================] - 9s 9ms/step - loss: 0.4151
32_x2_0.2dropout_